# 3-Quellen Datenfusion: VDEH + DNB + LoC

**Ziel:** Vollständige Fusion aller bibliographischen Daten aus drei Quellen

## Datenquellen
- **VDEH**: Basis-Katalogdaten (58.305 Datensätze)
- **DNB**: Deutsche Nationalbibliothek Anreicherung
- **LoC**: Library of Congress Anreicherung

## Fusion-Strategie
- 6 Varianten: DNB-ID (A), DNB-TA (B), DNB-TY (C), LoC-ID (D), LoC-TA (E), LoC-TY (F)
- Sprachbasierte Priorisierung (DNB für Deutsch, LoC für Englisch)
- KI-gestützte Konfliktauflösung
- Vollständiges Tracking aller Entscheidungen

In [1]:
# 🛠️ SETUP
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import json

# Add src to path
project_root = Path.cwd()
while not (project_root / 'config.yaml').exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root / 'src'))

from utils.notebook_utils import setup_notebook
from fusion import FusionEngine, OllamaClient

project_root, config = setup_notebook()
print(f"✅ Project root: {project_root}")
print(f"✅ Project: {config.get('project.name')} v{config.get('project.version')}")

2026-01-02 12:35:09 - utils.notebook_utils - INFO - Searching for project root...
2026-01-02 12:35:09 - utils.notebook_utils - INFO - Project root found: /media/sz/Data/Bibo/analysis
2026-01-02 12:35:09 - utils.notebook_utils - INFO - Loading configuration...
2026-01-02 12:35:09 - config_loader - INFO - Configuration loaded from /media/sz/Data/Bibo/analysis/config.yaml
2026-01-02 12:35:09 - utils.notebook_utils - INFO - Configuration loaded successfully: Dual-Source Bibliothek Bestandsvergleich


✅ Project root: /media/sz/Data/Bibo/analysis
✅ Project: Dual-Source Bibliothek Bestandsvergleich v2.2.0


In [ ]:
# 📂 LOAD DATA
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')

# Load VDEH base data (with language detection)
vdeh_path = processed_dir / '03_language_detected_data.parquet'
df_vdeh = pd.read_parquet(vdeh_path)
print(f"📂 VDEH: {len(df_vdeh):,} records")

# Load DNB enriched data
dnb_path = processed_dir / '04_dnb_enriched_data.parquet'
df_dnb = pd.read_parquet(dnb_path)
print(f"📂 DNB:  {len(df_dnb):,} records")

# Load LoC enriched data
loc_path = processed_dir / '04b_loc_enriched_data.parquet'
df_loc = pd.read_parquet(loc_path)
print(f"📂 LoC:  {len(df_loc):,} records")

# Check if record counts match
if len(df_vdeh) != len(df_dnb):
    print(f"\n⚠️  WARNING: DNB has {len(df_dnb)} records vs VDEH's {len(df_vdeh)}")
    print(f"   Trimming DNB to match VDEH length...")
    df_dnb = df_dnb.iloc[:len(df_vdeh)]
    
if len(df_vdeh) != len(df_loc):
    print(f"\n⚠️  WARNING: LoC has {len(df_loc)} records vs VDEH's {len(df_vdeh)}")
    print(f"   Trimming LoC to match VDEH length...")
    df_loc = df_loc.iloc[:len(df_vdeh)]

# Reset indices to ensure alignment
print(f"\n🔧 Aligning indices...")
df_vdeh = df_vdeh.reset_index(drop=True)
df_dnb = df_dnb.reset_index(drop=True)
df_loc = df_loc.reset_index(drop=True)

# Verify indices match
assert df_vdeh.index.equals(df_dnb.index), "DNB indices don't match VDEH"
assert df_vdeh.index.equals(df_loc.index), "LoC indices don't match VDEH"
print(f"✅ All indices match - {len(df_vdeh):,} records each")

In [ ]:
# 📊 DATA OVERVIEW
print("📊 Datenübersicht:\n")

# DNB coverage
dnb_cols = ['dnb_title', 'dnb_title_ta', 'dnb_title_ty']
has_dnb = df_dnb[dnb_cols].notna().any(axis=1)
print(f"DNB Anreicherung:")
print(f"   Datensätze mit DNB: {has_dnb.sum():,} ({has_dnb.sum()/len(df_dnb)*100:.1f}%)")
print(f"   - ID-basiert:  {df_dnb['dnb_title'].notna().sum():,}")
print(f"   - Title/Author: {df_dnb['dnb_title_ta'].notna().sum():,}")
print(f"   - Title/Year:   {df_dnb['dnb_title_ty'].notna().sum():,}")

# LoC coverage
loc_cols = ['loc_title', 'loc_title_ta', 'loc_title_ty']
has_loc = df_loc[loc_cols].notna().any(axis=1)
print(f"\nLoC Anreicherung:")
print(f"   Datensätze mit LoC: {has_loc.sum():,} ({has_loc.sum()/len(df_loc)*100:.1f}%)")
print(f"   - ID-basiert:  {df_loc['loc_title'].notna().sum():,}")
print(f"   - Title/Author: {df_loc['loc_title_ta'].notna().sum():,}")
print(f"   - Title/Year:   {df_loc['loc_title_ty'].notna().sum():,}")

# Combined
has_both = has_dnb & has_loc
has_either = has_dnb | has_loc
print(f"\nKombiniert:")
print(f"   Mit DNB oder LoC: {has_either.sum():,} ({has_either.sum()/len(df_vdeh)*100:.1f}%)")
print(f"   Mit DNB und LoC:  {has_both.sum():,} ({has_both.sum()/len(df_vdeh)*100:.1f}%)")
print(f"   Nur VDEH:         {(~has_either).sum():,} ({(~has_either).sum()/len(df_vdeh)*100:.1f}%)")

# Language distribution
print(f"\n🌍 Sprachverteilung:")
lang_dist = df_vdeh['detected_language'].value_counts()
for lang, count in lang_dist.head(5).items():
    print(f"   {lang}: {count:,} ({count/len(df_vdeh)*100:.1f}%)")

In [ ]:
# 🔄 MERGE DATA SOURCES
print("🔄 Zusammenführen der Datenquellen...\n")

# DNB columns to merge (DNB has no pages field)
dnb_merge_cols = [
    # ID variant
    'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher', 'dnb_isbn', 'dnb_issn',
    # Title/Author variant
    'dnb_title_ta', 'dnb_authors_ta', 'dnb_year_ta', 'dnb_publisher_ta', 'dnb_isbn_ta', 'dnb_issn_ta',
    # Title/Year variant
    'dnb_title_ty', 'dnb_authors_ty', 'dnb_year_ty', 'dnb_publisher_ty', 'dnb_isbn_ty', 'dnb_issn_ty',
    # Metadata
    'dnb_query_method'
]

# LoC columns to merge
loc_merge_cols = [
    # ID variant
    'loc_title', 'loc_authors', 'loc_year', 'loc_publisher', 'loc_isbn', 'loc_issn', 'loc_pages',
    # Title/Author variant
    'loc_title_ta', 'loc_authors_ta', 'loc_year_ta', 'loc_publisher_ta', 'loc_isbn_ta', 'loc_issn_ta', 'loc_pages_ta',
    # Title/Year variant
    'loc_title_ty', 'loc_authors_ty', 'loc_year_ty', 'loc_publisher_ty', 'loc_isbn_ty', 'loc_issn_ty', 'loc_pages_ty',
    # Metadata
    'loc_query_method'
]

# Merge all sources
df_merged = df_vdeh.copy()
df_merged = df_merged.join(df_dnb[dnb_merge_cols], how='left')
df_merged = df_merged.join(df_loc[loc_merge_cols], how='left')

print(f"✅ Merged dataset: {len(df_merged):,} records, {len(df_merged.columns)} columns")
print(f"\n📋 Columns: {df_merged.columns.tolist()[:10]}...")

In [ ]:
# 🤖 INITIALIZE FUSION ENGINE
print("🤖 Initialisiere FusionEngine...\n")

# Initialize Ollama client
ollama_client = OllamaClient(
    api_url="http://localhost:11434/api/generate",
    model="llama3.3:70b",
    timeout_sec=220,
    max_retries=4
)

# Test connection
if ollama_client.test_connection():
    print(f"✅ Ollama verbunden: {ollama_client.model}")
else:
    raise RuntimeError("❌ Ollama nicht erreichbar")

# Initialize FusionEngine
engine = FusionEngine(
    ollama_client=ollama_client,
    enable_loc=True,  # Enable LoC fusion
    ty_similarity_threshold=0.7
)

print(f"\n✅ FusionEngine initialisiert")
print(f"   - 3-Quellen-Fusion: VDEH + DNB + LoC")
print(f"   - 6 Varianten: A-F (DNB-ID/TA/TY, LoC-ID/TA/TY)")
print(f"   - Sprachbasierte Priorisierung")
print(f"   - Vollständiges Tracking")

In [ ]:
# 📦 CHECK FOR EXISTING CHECKPOINTS
print("📦 Prüfe auf bestehende Checkpoints...\n")

# Find all checkpoint files
checkpoint_files = sorted(processed_dir.glob('06_fusion_checkpoint_*.parquet'))

if checkpoint_files:
    # Get the latest checkpoint by extracting the record count from filename
    checkpoint_info = []
    for cp_file in checkpoint_files:
        # Extract number from filename: 06_fusion_checkpoint_15000.parquet -> 15000
        num_records = int(cp_file.stem.split('_')[-1])
        checkpoint_info.append((num_records, cp_file))
    
    # Sort by record count and get the latest
    checkpoint_info.sort(reverse=True)
    latest_count, latest_checkpoint = checkpoint_info[0]
    
    print(f"✅ Gefundene Checkpoints: {len(checkpoint_files)}")
    print(f"   Neuester Checkpoint: {latest_checkpoint.name}")
    print(f"   Datensätze: {latest_count:,} / {len(df_merged):,} ({latest_count/len(df_merged)*100:.1f}%)")
    print(f"   Verbleibend: {len(df_merged) - latest_count:,}")
    
    # Ask user if they want to resume
    resume_from_checkpoint = True  # Set to False to start fresh
    
    if resume_from_checkpoint:
        print(f"\n🔄 Lade Checkpoint: {latest_checkpoint.name}")
        df_checkpoint = pd.read_parquet(latest_checkpoint)
        
        # Convert checkpoint to results list
        results = df_checkpoint.to_dict('records')
        start_idx = latest_count
        
        # Calculate stats from checkpoint
        stats_from_checkpoint = {
            'neither': (df_checkpoint['title_source'] == 'vdeh').sum(),
            'only_dnb': df_checkpoint['title_source'].str.contains('dnb', na=False).sum(),
            'only_loc': df_checkpoint['title_source'].str.contains('loc', na=False).sum(),
            'both': df_checkpoint['fusion_selected_variant'].notna().sum(),
            'errors': df_checkpoint['rejection_reason'].notna().sum()
        }
        
        print(f"\n✅ Checkpoint geladen!")
        print(f"   Bereits verarbeitet: {len(results):,} Datensätze")
        print(f"   Statistik vom Checkpoint:")
        print(f"      Fast (VDEH):  {stats_from_checkpoint['neither']:,}")
        print(f"      Nur DNB:      {stats_from_checkpoint['only_dnb']:,}")
        print(f"      Nur LoC:      {stats_from_checkpoint['only_loc']:,}")
        print(f"      DNB + LoC:    {stats_from_checkpoint['both']:,}")
        print(f"      Fehler:       {stats_from_checkpoint['errors']:,}")
        print(f"\n⏭️  Starte ab Datensatz {start_idx + 1}")
    else:
        print(f"\n🆕 Starte von vorne (Checkpoint wird ignoriert)")
        results = []
        start_idx = 0
        stats_from_checkpoint = {'neither': 0, 'only_dnb': 0, 'only_loc': 0, 'both': 0, 'errors': 0}
else:
    print("ℹ️  Keine Checkpoints gefunden - starte von vorne")
    results = []
    start_idx = 0
    stats_from_checkpoint = {'neither': 0, 'only_dnb': 0, 'only_loc': 0, 'both': 0, 'errors': 0}

print(f"\n{'='*80}\n")

In [ ]:
# 🔥 RUN FUSION (WITH CHECKPOINT RESUME)
if 'start_idx' not in locals():
    # Fallback if cell 6 wasn't run
    results = []
    start_idx = 0
    stats_from_checkpoint = {'neither': 0, 'only_dnb': 0, 'only_loc': 0, 'both': 0, 'errors': 0}

if start_idx == 0:
    print("🔥 Starte Datenfusion (Optimiert)...\n")
else:
    print(f"🔥 Setze Datenfusion fort ab Datensatz {start_idx + 1}...\n")

print(f"Verarbeite {len(df_merged):,} Datensätze (noch {len(df_merged) - start_idx:,} zu verarbeiten)")
print(f"Startzeit: {datetime.now().strftime('%H:%M:%S')}")

# Pre-calculate which records need processing
dnb_cols = ['dnb_title', 'dnb_title_ta', 'dnb_title_ty']
has_dnb = df_merged[dnb_cols].notna().any(axis=1)

loc_cols = ['loc_title', 'loc_title_ta', 'loc_title_ty']
has_loc = df_merged[loc_cols].notna().any(axis=1)

has_both = has_dnb & has_loc
has_only_dnb = has_dnb & ~has_loc
has_only_loc = has_loc & ~has_dnb
has_neither = ~has_dnb & ~has_loc

if start_idx == 0:
    print(f"\n📊 Verarbeitungsstrategie:")
    print(f"   Keine Anreicherung (nur VDEH):  {has_neither.sum():,} ({has_neither.sum()/len(df_merged)*100:.1f}%) - Fast Copy")
    print(f"   Nur DNB (auto select):          {has_only_dnb.sum():,} ({has_only_dnb.sum()/len(df_merged)*100:.1f}%) - Simple Merge")
    print(f"   Nur LoC (auto select):          {has_only_loc.sum():,} ({has_only_loc.sum()/len(df_merged)*100:.1f}%) - Simple Merge")
    print(f"   DNB + LoC (KI entscheidet):     {has_both.sum():,} ({has_both.sum()/len(df_merged)*100:.1f}%) - Full Engine")

print(f"\n{'='*80}\n")

# Helper function for simple merge (no AI needed)
def simple_merge_dnb(row, vdeh_data):
    """Fast merge when only DNB data available."""
    # Prefer ID-based, then TA, then TY
    if pd.notna(row.get('dnb_title')):
        return {
            **vdeh_data,
            'title': row.get('dnb_title', vdeh_data['title']),
            'authors': row.get('dnb_authors', vdeh_data['authors']),
            'year': row.get('dnb_year', vdeh_data['year']),
            'publisher': row.get('dnb_publisher', vdeh_data['publisher']),
            'isbn': row.get('dnb_isbn', vdeh_data['isbn']),
            'issn': row.get('dnb_issn', vdeh_data['issn']),
            'title_source': 'dnb_id',
            'authors_source': 'dnb_id',
            'year_source': 'dnb_id',
            'publisher_source': 'dnb_id',
            'pages_source': 'vdeh',
            'isbn_source': 'dnb_id' if pd.notna(row.get('dnb_isbn')) else 'vdeh',
            'issn_source': 'dnb_id' if pd.notna(row.get('dnb_issn')) else 'vdeh',
            'dnb_variant_selected': 'dnb_id'
        }
    elif pd.notna(row.get('dnb_title_ta')):
        return {
            **vdeh_data,
            'title': row.get('dnb_title_ta', vdeh_data['title']),
            'authors': row.get('dnb_authors_ta', vdeh_data['authors']),
            'year': row.get('dnb_year_ta', vdeh_data['year']),
            'publisher': row.get('dnb_publisher_ta', vdeh_data['publisher']),
            'isbn': row.get('dnb_isbn_ta', vdeh_data['isbn']),
            'issn': row.get('dnb_issn_ta', vdeh_data['issn']),
            'title_source': 'dnb_title_author',
            'authors_source': 'dnb_title_author',
            'year_source': 'dnb_title_author',
            'publisher_source': 'dnb_title_author',
            'pages_source': 'vdeh',
            'isbn_source': 'dnb_title_author' if pd.notna(row.get('dnb_isbn_ta')) else 'vdeh',
            'issn_source': 'dnb_title_author' if pd.notna(row.get('dnb_issn_ta')) else 'vdeh',
            'dnb_variant_selected': 'dnb_title_author'
        }
    else:
        return {
            **vdeh_data,
            'title': row.get('dnb_title_ty', vdeh_data['title']),
            'authors': row.get('dnb_authors_ty', vdeh_data['authors']),
            'year': row.get('dnb_year_ty', vdeh_data['year']),
            'publisher': row.get('dnb_publisher_ty', vdeh_data['publisher']),
            'isbn': row.get('dnb_isbn_ty', vdeh_data['isbn']),
            'issn': row.get('dnb_issn_ty', vdeh_data['issn']),
            'title_source': 'dnb_title_year',
            'authors_source': 'dnb_title_year',
            'year_source': 'dnb_title_year',
            'publisher_source': 'dnb_title_year',
            'pages_source': 'vdeh',
            'isbn_source': 'dnb_title_year' if pd.notna(row.get('dnb_isbn_ty')) else 'vdeh',
            'issn_source': 'dnb_title_year' if pd.notna(row.get('dnb_issn_ty')) else 'vdeh',
            'dnb_variant_selected': 'dnb_title_year'
        }

def simple_merge_loc(row, vdeh_data):
    """Fast merge when only LoC data available."""
    # Prefer ID-based, then TA, then TY
    if pd.notna(row.get('loc_title')):
        return {
            **vdeh_data,
            'title': row.get('loc_title', vdeh_data['title']),
            'authors': row.get('loc_authors', vdeh_data['authors']),
            'year': row.get('loc_year', vdeh_data['year']),
            'publisher': row.get('loc_publisher', vdeh_data['publisher']),
            'pages': row.get('loc_pages', vdeh_data['pages']),
            'isbn': row.get('loc_isbn', vdeh_data['isbn']),
            'issn': row.get('loc_issn', vdeh_data['issn']),
            'title_source': 'loc_id',
            'authors_source': 'loc_id',
            'year_source': 'loc_id',
            'publisher_source': 'loc_id',
            'pages_source': 'loc_id' if pd.notna(row.get('loc_pages')) else 'vdeh',
            'isbn_source': 'loc_id' if pd.notna(row.get('loc_isbn')) else 'vdeh',
            'issn_source': 'loc_id' if pd.notna(row.get('loc_issn')) else 'vdeh'
        }
    elif pd.notna(row.get('loc_title_ta')):
        return {
            **vdeh_data,
            'title': row.get('loc_title_ta', vdeh_data['title']),
            'authors': row.get('loc_authors_ta', vdeh_data['authors']),
            'year': row.get('loc_year_ta', vdeh_data['year']),
            'publisher': row.get('loc_publisher_ta', vdeh_data['publisher']),
            'pages': row.get('loc_pages_ta', vdeh_data['pages']),
            'isbn': row.get('loc_isbn_ta', vdeh_data['isbn']),
            'issn': row.get('loc_issn_ta', vdeh_data['issn']),
            'title_source': 'loc_title_author',
            'authors_source': 'loc_title_author',
            'year_source': 'loc_title_author',
            'publisher_source': 'loc_title_author',
            'pages_source': 'loc_title_author' if pd.notna(row.get('loc_pages_ta')) else 'vdeh',
            'isbn_source': 'loc_title_author' if pd.notna(row.get('loc_isbn_ta')) else 'vdeh',
            'issn_source': 'loc_title_author' if pd.notna(row.get('loc_issn_ta')) else 'vdeh'
        }
    else:
        return {
            **vdeh_data,
            'title': row.get('loc_title_ty', vdeh_data['title']),
            'authors': row.get('loc_authors_ty', vdeh_data['authors']),
            'year': row.get('loc_year_ty', vdeh_data['year']),
            'publisher': row.get('loc_publisher_ty', vdeh_data['publisher']),
            'pages': row.get('loc_pages_ty', vdeh_data['pages']),
            'isbn': row.get('loc_isbn_ty', vdeh_data['isbn']),
            'issn': row.get('loc_issn', vdeh_data['issn']),
            'title_source': 'loc_title_year',
            'authors_source': 'loc_title_year',
            'year_source': 'loc_title_year',
            'publisher_source': 'loc_title_year',
            'pages_source': 'loc_title_year' if pd.notna(row.get('loc_pages_ty')) else 'vdeh',
            'isbn_source': 'loc_title_year' if pd.notna(row.get('loc_isbn_ty')) else 'vdeh',
            'issn_source': 'loc_title_year' if pd.notna(row.get('loc_issn_ty')) else 'vdeh'
        }

# Initialize tracking
start_time = datetime.now()
checkpoint_interval = 5000
stats = stats_from_checkpoint.copy()
last_progress_time = start_time

# Process records starting from start_idx
for i, (idx, row) in enumerate(df_merged.iloc[start_idx:].iterrows()):
    # VDEH base data - NOW INCLUDING ISBN/ISSN
    vdeh_data = {
        'title': row.get('title'),
        'authors': row.get('authors_str'),
        'year': row.get('year'),
        'publisher': row.get('publisher'),
        'pages': row.get('pages'),
        'isbn': row.get('isbn'),
        'issn': row.get('issn'),
        'title_source': 'vdeh',
        'authors_source': 'vdeh',
        'year_source': 'vdeh',
        'publisher_source': 'vdeh',
        'pages_source': 'vdeh',
        'isbn_source': 'vdeh' if pd.notna(row.get('isbn')) else None,
        'issn_source': 'vdeh' if pd.notna(row.get('issn')) else None
    }
    
    # Route based on data availability
    if has_neither.loc[idx]:
        # Fast path: No enrichment
        results.append(vdeh_data)
        stats['neither'] += 1
        
    elif has_only_dnb.loc[idx]:
        # Simple merge: Only DNB
        results.append(simple_merge_dnb(row, vdeh_data))
        stats['only_dnb'] += 1
        
    elif has_only_loc.loc[idx]:
        # Simple merge: Only LoC
        results.append(simple_merge_loc(row, vdeh_data))
        stats['only_loc'] += 1
        
    else:
        # Complex case: Both DNB and LoC - use full engine
        try:
            result = engine.merge_record(row)
            results.append(result.to_dict())
            stats['both'] += 1
            
            # Detailed progress for AI decisions every 10 records
            if stats['both'] % 10 == 0:
                elapsed = (datetime.now() - start_time).total_seconds()
                ai_since_checkpoint = stats['both'] - stats_from_checkpoint['both']
                rate = ai_since_checkpoint / elapsed if elapsed > 0 else 0
                remaining_ai = has_both.sum() - stats['both']
                eta = remaining_ai / rate if rate > 0 else 0
                print(f"\r🤖 AI: [{stats['both']}/{has_both.sum()}] {stats['both']/has_both.sum()*100:.1f}% | "
                      f"{rate:.2f} rec/s | ETA: {eta/60:.0f}min | "
                      f"Total: {len(results):,}/{len(df_merged):,}", end='', flush=True)
                
        except Exception as e:
            print(f"\n❌ ERROR at {idx}: {e}")
            results.append({**vdeh_data, 'title_source': 'error', 'rejection_reason': str(e)})
            stats['errors'] += 1
    
    # Progress update every 1000 records (more frequent!)
    if len(results) % 1000 == 0 and len(results) > start_idx:
        now = datetime.now()
        total_elapsed = (now - start_time).total_seconds()
        records_since_checkpoint = len(results) - start_idx
        overall_rate = records_since_checkpoint / total_elapsed if total_elapsed > 0 else 0
        overall_eta = (len(df_merged) - len(results)) / overall_rate if overall_rate > 0 else 0
        
        print(f"\n⏱️  [{now.strftime('%H:%M:%S')}] Progress: {len(results):,}/{len(df_merged):,} ({len(results)/len(df_merged)*100:.1f}%)")
        print(f"   Rate: {overall_rate:.0f} rec/s | ETA: {overall_eta/60:.0f}min | Elapsed: {total_elapsed/60:.1f}min")
        print(f"   ✓ VDEH: {stats['neither']:,} | DNB: {stats['only_dnb']:,} | LoC: {stats['only_loc']:,} | AI: {stats['both']:,}")
    
    # Detailed summary every 5000 records
    if len(results) % 5000 == 0 and len(results) > start_idx:
        total_elapsed = (datetime.now() - start_time).total_seconds()
        records_since_checkpoint = len(results) - start_idx
        overall_rate = records_since_checkpoint / total_elapsed if total_elapsed > 0 else 0
        overall_eta = (len(df_merged) - len(results)) / overall_rate if overall_rate > 0 else 0
        print(f"\n{'='*80}")
        print(f"📊 Checkpoint-Fortschritt: [{len(results):,}/{len(df_merged):,}] {len(results)/len(df_merged)*100:.1f}%")
        print(f"   Gesamt-Rate: {overall_rate:.0f} rec/s | ETA: {overall_eta/60:.0f}min")
        print(f"   Fast (VDEH):  {stats['neither']:,}")
        print(f"   Nur DNB:      {stats['only_dnb']:,}")
        print(f"   Nur LoC:      {stats['only_loc']:,}")
        print(f"   DNB + LoC:    {stats['both']:,}")
        print(f"   Fehler:       {stats['errors']:,}")
        print(f"{'='*80}")
    
    # Checkpoint save
    if len(results) % checkpoint_interval == 0 and len(results) > start_idx:
        checkpoint_path = processed_dir / f'06_fusion_checkpoint_{len(results)}.parquet'
        df_checkpoint = pd.DataFrame(results, index=df_merged.index[:len(results)])
        df_checkpoint.to_parquet(checkpoint_path)
        print(f"\n💾 Checkpoint gespeichert: {checkpoint_path.name}")

print(f"\n\n{'='*80}")
print(f"✅ Fusion abgeschlossen!")
print(f"   Keine Anreicherung: {stats['neither']:,}")
print(f"   Nur DNB:            {stats['only_dnb']:,}")
print(f"   Nur LoC:            {stats['only_loc']:,}")
print(f"   DNB + LoC (KI):     {stats['both']:,}")
print(f"   Fehler:             {stats['errors']:,}")
print(f"Endzeit: {datetime.now().strftime('%H:%M:%S')}")
elapsed_total = (datetime.now() - start_time).total_seconds()
print(f"Dauer: {elapsed_total/60:.1f} Minuten")
if start_idx > 0:
    print(f"Verarbeitet: {len(results) - start_idx:,} neue Datensätze")

In [ ]:
# 📊 CREATE RESULT DATAFRAME
print("📊 Erstelle Ergebnis-DataFrame...\n")

df_fused = pd.DataFrame(results, index=df_merged.index)

print(f"✅ Fused dataset: {len(df_fused):,} records, {len(df_fused.columns)} columns")
print(f"\n📋 Result columns:")
for col in df_fused.columns:
    non_null = df_fused[col].notna().sum()
    print(f"   {col:35s}: {non_null:,} ({non_null/len(df_fused)*100:.1f}%)")

In [ ]:
# 📈 FUSION STATISTICS
print("📈 Fusion Statistiken:\n")

# Source distribution
print("Datenquellen-Verteilung:")
source_dist = df_fused['title_source'].value_counts()
for source, count in source_dist.items():
    print(f"   {source:20s}: {count:,} ({count/len(df_fused)*100:.1f}%)")

# AI usage
print(f"\nKI-Nutzung:")
ai_decisions = df_fused['fusion_selected_variant'].notna().sum()
print(f"   KI-Entscheidungen: {ai_decisions:,} ({ai_decisions/len(df_fused)*100:.1f}%)")

if ai_decisions > 0:
    variant_dist = df_fused['fusion_selected_variant'].value_counts()
    print(f"\n   Varianten-Auswahl:")
    variant_names = {
        'A': 'DNB-ID', 'B': 'DNB-TA', 'C': 'DNB-TY',
        'D': 'LoC-ID', 'E': 'LoC-TA', 'F': 'LoC-TY',
        'KEINE': 'Abgelehnt'
    }
    for variant, count in variant_dist.items():
        variant_name = variant_names.get(variant, variant)
        print(f"      {variant} ({variant_name:12s}): {count:,} ({count/ai_decisions*100:.1f}%)")

# Conflicts detected
conflicts_detected = df_fused['fusion_conflicts_detected'].notna().sum()
print(f"\nKonflikte:")
print(f"   Datensätze mit Konflikten: {conflicts_detected:,} ({conflicts_detected/len(df_fused)*100:.1f}%)")

# Rejections
rejections = df_fused['rejection_reason'].notna().sum()
print(f"\nAblehnungen:")
print(f"   Abgelehnte Matches: {rejections:,} ({rejections/len(df_fused)*100:.1f}%)")

In [ ]:
# 📊 QUALITY METRICS
print("📊 Qualitätsmetriken:\n")

# Data completeness
print("Datenvollständigkeit:")
for field in ['title', 'authors', 'year', 'publisher', 'pages']:
    complete = df_fused[field].notna().sum()
    print(f"   {field:15s}: {complete:,} ({complete/len(df_fused)*100:.1f}%)")

# Title similarity scores (where available)
if 'title_similarity_score' in df_fused.columns:
    scores = df_fused['title_similarity_score'].dropna()
    if len(scores) > 0:
        print(f"\nTitel-Ähnlichkeit (bei {len(scores):,} Matches):")
        print(f"   Durchschnitt: {scores.mean():.1%}")
        print(f"   Median:       {scores.median():.1%}")
        print(f"   Min:          {scores.min():.1%}")
        print(f"   Max:          {scores.max():.1%}")

# Language-based source selection
print(f"\nSprachbasierte Quellenwahl:")
lang_source = df_merged[['detected_language']].join(df_fused[['title_source']])
lang_source_cross = pd.crosstab(
    lang_source['detected_language'], 
    lang_source['title_source'],
    normalize='index'
)
print(lang_source_cross.head(10).to_string())

In [ ]:
# 💾 SAVE RESULTS
print("💾 Speichere Ergebnisse...\n")

# Save fused data
output_path = processed_dir / '06_vdeh_dnb_loc_fused_data.parquet'
df_fused.to_parquet(output_path, index=True)
print(f"✅ Fused data saved: {output_path}")
print(f"   Size: {output_path.stat().st_size / 1024 / 1024:.1f} MB")

# Save fusion statistics
stats = {
    'total_records': len(df_fused),
    'timestamp': datetime.now().isoformat(),
    'sources': source_dist.to_dict(),
    'ai_decisions': int(ai_decisions),
    'conflicts_detected': int(conflicts_detected),
    'rejections': int(rejections),
    'completeness': {
        field: int(df_fused[field].notna().sum())
        for field in ['title', 'authors', 'year', 'publisher', 'pages']
    }
}

stats_path = processed_dir / '06_fusion_statistics.json'
with open(stats_path, 'w', encoding='utf-8') as f:
    json.dump(stats, f, indent=2, ensure_ascii=False)
print(f"✅ Statistics saved: {stats_path}")

# Clean up checkpoints
checkpoint_files = list(processed_dir.glob('06_fusion_checkpoint_*.parquet'))
if checkpoint_files:
    print(f"\n🗑️  Cleaning up {len(checkpoint_files)} checkpoint files...")
    for cp_file in checkpoint_files:
        cp_file.unlink()
    print(f"✅ Checkpoints removed")

print(f"\n{'='*80}")
print(f"✅ FUSION COMPLETE!")
print(f"{'='*80}")

In [ ]:
# 🔍 SAMPLE INSPECTION
print("🔍 Stichproben der fusionierten Daten:\n")

# Show a few examples of different source types
print("="*80)
print("Beispiel 1: VDEH Original (kein Match)")
print("="*80)
vdeh_only = df_fused[df_fused['title_source'] == 'vdeh'].head(1)
if len(vdeh_only) > 0:
    idx = vdeh_only.index[0]
    print(f"Index: {idx}")
    print(f"Title:     {vdeh_only.loc[idx, 'title']}")
    print(f"Authors:   {vdeh_only.loc[idx, 'authors']}")
    print(f"Year:      {vdeh_only.loc[idx, 'year']}")
    print(f"Source:    {vdeh_only.loc[idx, 'title_source']}")

print(f"\n{'='*80}")
print("Beispiel 2: DNB Match")
print("="*80)
dnb_match = df_fused[df_fused['title_source'].str.contains('dnb', na=False)].head(1)
if len(dnb_match) > 0:
    idx = dnb_match.index[0]
    print(f"Index: {idx}")
    print(f"Title:     {dnb_match.loc[idx, 'title']}")
    print(f"Authors:   {dnb_match.loc[idx, 'authors']}")
    print(f"Year:      {dnb_match.loc[idx, 'year']}")
    print(f"Source:    {dnb_match.loc[idx, 'title_source']}")
    if pd.notna(dnb_match.loc[idx, 'title_similarity_score']):
        print(f"Similarity: {dnb_match.loc[idx, 'title_similarity_score']:.1%}")

print(f"\n{'='*80}")
print("Beispiel 3: LoC Match")
print("="*80)
loc_match = df_fused[df_fused['title_source'].str.contains('loc', na=False)].head(1)
if len(loc_match) > 0:
    idx = loc_match.index[0]
    print(f"Index: {idx}")
    print(f"Title:     {loc_match.loc[idx, 'title']}")
    print(f"Authors:   {loc_match.loc[idx, 'authors']}")
    print(f"Year:      {loc_match.loc[idx, 'year']}")
    print(f"Source:    {loc_match.loc[idx, 'title_source']}")
    if pd.notna(loc_match.loc[idx, 'title_similarity_score']):
        print(f"Similarity: {loc_match.loc[idx, 'title_similarity_score']:.1%}")

print(f"\n{'='*80}")
print("Beispiel 4: KI-Entscheidung")
print("="*80)
ai_decision = df_fused[df_fused['fusion_selected_variant'].notna()].head(1)
if len(ai_decision) > 0:
    idx = ai_decision.index[0]
    print(f"Index: {idx}")
    print(f"Title:     {ai_decision.loc[idx, 'title']}")
    print(f"Authors:   {ai_decision.loc[idx, 'authors']}")
    print(f"Year:      {ai_decision.loc[idx, 'year']}")
    print(f"Source:    {ai_decision.loc[idx, 'title_source']}")
    print(f"Variant:   {ai_decision.loc[idx, 'fusion_selected_variant']}")
    if pd.notna(ai_decision.loc[idx, 'ai_reasoning']):
        print(f"Reasoning: {ai_decision.loc[idx, 'ai_reasoning'][:150]}...")